In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import re
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

Using TensorFlow backend.


In [2]:
data=pd.read_csv("data2.csv")
print(type(data["Class"]))
data['Tweet']=data['Tweet'].apply(lambda x: x.lower())
data.loc[data["Class"]==0]




<class 'pandas.core.series.Series'>


,no,Tweet,Class
0,1,",rt @sza: its like w less ppl outside we all ...",0
1,2,",rt @realrarety: hey everybody imma dirty bitch",0
2,3,",rt @aria66369199: do not mess with a girl of...",0
4,5,",rt @faiza_baby: this is so so sad. how is an...",0
5,6,",@kofoworola__a happy birthday tout damsel wi...",0
...,...,...,...
989,990,",just tested niggative",0
990,991,",@urmomfavplayer yeah well trash won two supe...",0
991,992,",rt @plkajin: rt and ill guess if ur het or n...",0
998,999,",i'm bout to go live",0


In [3]:
data['Tweet']=data['Tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
for idx,row in data.iterrows():
    row[1] = row[1].replace('rt',' ')
data['Tweet']

0       rt sza its like w less ppl outside we all hav...
1           rt realrarety hey everybody imma dirty bitch
2       rt aria66369199 do not mess with a girl of sh...
3                         rydo90 same httpstcomimjh0kky2
4       rt faiza_baby this is so so sad how is anyone...
                             ...                        
995     rt qikipedia you are more likely to be killed...
996     this sounds fucking wild and horrific release...
997                                         film and art
998                                   im bout to go live
999     rt gabimorinn idk how to explain this but thi...
Name: Tweet, Length: 1000, dtype: object

In [4]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
#tokenizer.fit_on_texts(data[:1000].values)
tokenizer.fit_on_texts(data['Tweet'].values)
X = tokenizer.texts_to_sequences(data['Tweet'].values)
X = pad_sequences(X)


In [5]:
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 51, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 51, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = pd.get_dummies(data['Class']).values
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [1 0]
 [1 0]]
(800, 51) (800, 2)
(200, 51) (200, 2)


In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/7





 - 7s - loss: 0.6591 - acc: 0.6013
Epoch 2/7
 - 4s - loss: 0.6057 - acc: 0.6900
Epoch 3/7
 - 4s - loss: 0.4307 - acc: 0.8225
Epoch 4/7
 - 4s - loss: 0.3053 - acc: 0.8800
Epoch 5/7
 - 4s - loss: 0.2115 - acc: 0.9088
Epoch 6/7
 - 4s - loss: 0.1603 - acc: 0.9413
Epoch 7/7
 - 4s - loss: 0.1173 - acc: 0.9600


In [8]:
validation_size = 1500
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.88
acc: 0.66


In [9]:
predictions=model.predict(X_test,verbose=0)
predictions

array([[1.46623895e-01, 8.53376150e-01],
       [9.62490499e-01, 3.75095010e-02],
       [2.14575077e-04, 9.99785483e-01],
       [9.34767485e-01, 6.52324855e-02],
       [9.98738587e-01, 1.26147119e-03],
       [9.54052359e-02, 9.04594719e-01],
       [9.81832040e-04, 9.99018192e-01],
       [4.39462453e-01, 5.60537517e-01],
       [1.87207863e-01, 8.12792182e-01],
       [6.84274256e-01, 3.15725654e-01],
       [8.27361584e-01, 1.72638431e-01],
       [8.06141570e-02, 9.19385850e-01],
       [2.92504966e-01, 7.07495034e-01],
       [1.30232304e-01, 8.69767785e-01],
       [9.66585875e-01, 3.34141739e-02],
       [8.00897419e-01, 1.99102551e-01],
       [5.94410777e-01, 4.05589283e-01],
       [7.30282962e-01, 2.69716978e-01],
       [1.90623879e-01, 8.09376180e-01],
       [1.87290996e-01, 8.12709033e-01],
       [5.82076848e-01, 4.17923123e-01],
       [9.97655034e-01, 2.34498992e-03],
       [3.21703702e-01, 6.78296268e-01],
       [7.76068449e-01, 2.23931551e-01],
       [6.595503

In [10]:
p=np.argmax(predictions,axis=1)
p

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1])

In [11]:
Y_test=Y_test.argmax(axis=1)
# Y_test

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
print("Accuracy score: ", accuracy_score(Y_test, p))
print("Precision score: ", precision_score(Y_test, p))
print("Recall score: ", recall_score(Y_test, p))
print("Fl Score", f1_score(Y_test, p))

Accuracy score:  0.655
Precision score:  0.7297297297297297
Recall score:  0.675
Fl Score 0.7012987012987014
